In [8]:
%%writefile agro.l
%{
#include <stdio.h>
#include "parser.tab.h"
%}

digit           [0-9]
number          {digit}+
letter          [A-Za-z]
identifier      {letter}({letter}|{digit}|_)*
whitespace      [ \t\n]+

%%

"se"            { printf("SE\n"); return SE; }
"entao"         { printf("ENTAO\n"); return ENTAO; }
"senao"         { printf("SENAO\n"); return SENAO; }
"enquanto"      { printf("ENQUANTO\n"); return ENQUANTO; }
"comecar"       { printf("COMECAR\n"); return COMECAR; }
"terminar"      { printf("TERMINAR\n"); return TERMINAR; }
"plantar"       { printf("PLANTAR\n"); return PLANTAR; }
"colher"        { printf("COLHER\n"); return COLHER; }
"registrar"     { printf("REGISTRAR\n"); return REGISTRAR; }
"vacinar"       { printf("VACINAR\n"); return VACINAR; }
"em"            { printf("EM\n"); return EM; }
"gado"          { printf("GADO\n"); return GADO; }
"ovelha"        { printf("OVELHA\n"); return OVELHA; }
"porco"         { printf("PORCO\n"); return PORCO; }
"galinha"       { printf("GALINHA\n"); return GALINHA; }
{number}        { printf("NUMERO: %s\n", yytext); return NUMERO; }
{identifier}    { printf("IDENTIFICADOR: %s\n", yytext); return IDENTIFICADOR; }
"+"             { printf("MAIS\n"); return MAIS; }
"-"             { printf("MENOS\n"); return MENOS; }
"*"             { printf("MULTIPLICAR\n"); return MULTIPLICAR; }
"/"             { printf("DIVIDIR\n"); return DIVIDIR; }
"="             { printf("IGUAL\n"); return IGUAL; }
";"             { printf("PONTO_E_VIRGULA\n"); return PONTO_E_VIRGULA; }
"("             { printf("ABRE_PARENTESES\n"); return ABRE_PARENTESES; }
")"             { printf("FECHA_PARENTESES\n"); return FECHA_PARENTESES; }
{whitespace}    { /* ignora espaços em branco */ }
.               { printf("Caractere inesperado: %c\n", yytext[0]); }

%%

Overwriting agro.l


In [9]:
%%writefile parser.y
%{
#include <stdio.h>
#include <stdlib.h>
extern int yylex();
extern int yylineno;
void yyerror(const char *s);
%}

%token NUMERO IDENTIFICADOR SE ENTAO SENAO ENQUANTO COMECAR TERMINAR
%token MAIS MENOS MULTIPLICAR DIVIDIR IGUAL PONTO_E_VIRGULA
%token ABRE_PARENTESES FECHA_PARENTESES
%token PLANTAR COLHER REGISTRAR VACINAR EM
%token GADO OVELHA PORCO GALINHA

%nonassoc MENOR_QUE_SENAO
%nonassoc SENAO
%%
programa:
    | programa declaracao PONTO_E_VIRGULA
    ;

declaracao:
      atribuicao
    | estrutura_controle
    | bloco
    | operacao_cultivo
    | operacao_pecuaria
    ;

atribuicao:
    IDENTIFICADOR IGUAL expressao
    ;

expressao:
    termo
    | expressao MAIS termo
    | expressao MENOS termo
    ;

termo:
    fator
    | termo MULTIPLICAR fator
    | termo DIVIDIR fator
    ;

fator:
    NUMERO
    | IDENTIFICADOR
    | ABRE_PARENTESES expressao FECHA_PARENTESES
    ;

estrutura_controle:
      estrutura_se
    | estrutura_enquanto
    ;

estrutura_se:
    SE ABRE_PARENTESES expressao FECHA_PARENTESES ENTAO declaracao %prec MENOR_QUE_SENAO
    | SE ABRE_PARENTESES expressao FECHA_PARENTESES ENTAO declaracao SENAO declaracao
    ;

estrutura_enquanto:
    ENQUANTO ABRE_PARENTESES expressao FECHA_PARENTESES declaracao
    ;

bloco:
    COMECAR programa TERMINAR
    ;

operacao_cultivo:
      PLANTAR IDENTIFICADOR EM NUMERO
    | COLHER IDENTIFICADOR
    ;

operacao_pecuaria:
      REGISTRAR IDENTIFICADOR tipo_animal
    | VACINAR IDENTIFICADOR EM NUMERO
    ;

tipo_animal:
      GADO
    | OVELHA
    | PORCO
    | GALINHA
    ;

%%
void yyerror(const char *s) {
    fprintf(stderr, "Erro: %s na linha %d\n", s, yylineno);
}

int main(void) {
    yyparse();
    return 0;
}

Overwriting parser.y


In [10]:
%%writefile agro.txt

registrar animal1 gado;
plantar milho em 01012023;
se (1) entao
    vacinar animal1 em 02022023;
senao
    colher milho;

enquanto (1) comecar
    plantar trigo em 03032023;
terminar


Overwriting agro.txt


In [15]:
!sudo apt-get install bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  bison-doc
The following NEW packages will be installed:
  bison
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 748 kB of archives.
After this operation, 2,519 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 bison amd64 2:3.8.2+dfsg-1build1 [748 kB]
Fetched 748 kB in 2s (379 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package bison.
(Reading database ... 120

In [16]:
!sudo apt-get install flex

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfl-dev libfl2
Suggested packages:
  flex-doc
The following NEW packages will be installed:
  flex libfl-dev libfl2
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 324 kB of archives.
After this operation, 1,148 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 flex amd64 2.6.4-8build2 [307 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl2 amd64 2.6.4-8build2 [10.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl-dev amd64 2.6.4-8build2 [6,236 B]
Fetched 324 kB in 2s (189 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Read

In [17]:
!bison -d -Wcounterexamples parser.y

In [18]:
!flex -o lex.yy.c agro.l

In [19]:
!gcc -o agro lex.yy.c -lfl

In [21]:
!./agro < agro.txt

REGISTRAR
IDENTIFICADOR: animal1
GADO
PONTO_E_VIRGULA
PLANTAR
IDENTIFICADOR: milho
EM
NUMERO: 01012023
PONTO_E_VIRGULA
SE
ABRE_PARENTESES
NUMERO: 1
FECHA_PARENTESES
ENTAO
VACINAR
IDENTIFICADOR: animal1
EM
NUMERO: 02022023
PONTO_E_VIRGULA
SENAO
COLHER
IDENTIFICADOR: milho
PONTO_E_VIRGULA
ENQUANTO
ABRE_PARENTESES
NUMERO: 1
FECHA_PARENTESES
COMECAR
PLANTAR
IDENTIFICADOR: trigo
EM
NUMERO: 03032023
PONTO_E_VIRGULA
TERMINAR
